In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

In [2]:
data_train = pd.read_csv(r'..\data\train\data-fulltext-train.csv')
data_test = pd.read_csv(r'..\data\test\data-test.csv')

data_train.abstract = data_train.abstract.astype(str)
data_train.journal = data_train.journal.astype(str)
data_test.abstract = data_test.abstract.astype(str)
data_test.journal = data_test.journal.astype(str)

MRR

In [3]:
def MRR(preds, truths, n,labels):
    #print(preds)
    #print(np.argsort(preds))
    best_n = np.argsort(preds, axis=1)[:,-n:] #mejores items
    #print(preds.shape, truths.shape)
    #print(best_n,"-----",truths)
    #ts = np.argmax(truths, axis=1)
     
    if (preds.shape[1]<n):
        total =  preds.shape[1]-1
        #print("TTT",total-1)
        #print(ts)
    else:
        total = n

    successes = 0

    for i in range(len(truths)):
        indice = np.where(labels==truths[i])[0]
        #print( "i ",i, "= ",truths[i], " indice ", indice, "---- ",best_n[i])
        #print(best_n[i,:],  "-- ",indice)
        #print(" pos",pos, " ",len(pos))

        if indice in best_n[i]:
            pos = np.where(best_n[i]==indice)
            #print(pos[0]," ",total," val ",1.0/(total-pos[0]+1))
            successes += 1.0/(total-pos[0])
            
    #print(successes, "xxx", truths.shape[0])
    return float(successes)/len(truths)

Vectorizado TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

X_train = data_train['abstract']
y_train = data_train['journal']
X_test = data_test['abstract']
y_test = data_test['journal']

tfidf = TfidfVectorizer(min_df=20, max_df=0.9)
X_train_final = tfidf.fit_transform(X_train)
X_test_final = tfidf.transform(X_test)

In [5]:
len(tfidf.get_feature_names_out())

56588

In [ ]:
# # Pruebas MultinomialNB:

# from sklearn.naive_bayes import MultinomialNB

# model = MultinomialNB(alpha=0.004)
# print("Vamos a entrenar 1!")
# model.fit(X_train_final,y_train)
# print("1. Entrenamiento terminado. Vamos a la predicción!")

In [ ]:
# # Pruebas KNeighborsClassifier:

# from sklearn.neighbors import KNeighborsClassifier

# model = KNeighborsClassifier(n_neighbors=50, n_jobs=-1)
# print("Vamos a entrenar!")
# model.fit(X_train_final,y_train)
# print("1. Entrenamiento terminado. Vamos a la predicción!")

In [ ]:
# # Pruebas RandomForestClassifier:

# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier(n_estimators=150, max_features="sqrt", criterion="gini", max_depth=200, n_jobs=-1)
# print("Vamos a entrenar!")
# model.fit(X_train_final,y_train)
# print("1. Entrenamiento terminado. Vamos a la predicción!")

In [ ]:
# # Pruebas LogisticRegression:

# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(multi_class="multinomial", solver="saga", penalty="none", n_jobs=-1)
# print("Vamos a entrenar!")
# model.fit(X_train_final,y_train)
# print("1. Entrenamiento terminado. Vamos a la predicción!")

In [ ]:
# # Pruebas ComplementNB:

# from sklearn.naive_bayes import ComplementNB

# model = ComplementNB(alpha=0.004)
# print("Vamos a entrenar 1!")
# model.fit(X_train_final,y_train)
# print("1. Entrenamiento terminado. Vamos a la predicción!")

Accuracy Score

In [ ]:
from sklearn.metrics import top_k_accuracy_score

proba = model.predict_proba(X_test_final)

print('acc@1 = ',top_k_accuracy_score(y_test, proba,k=1,labels=np.sort(y_train.unique())))
print('acc@5 = ',top_k_accuracy_score(y_test, proba,k=5,labels=np.sort(y_train.unique())))
print('acc@10 = ',top_k_accuracy_score(y_test, proba,k=10,labels=np.sort(y_train.unique())))

In [ ]:
truths = y_test.tolist()

print("MRR@5: "+ str(MRR(proba, truths, 5, model.classes_)))
print("MRR@10: "+ str(MRR(proba, truths, 10, model.classes_)))